In [5]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [6]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [8]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [9]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [10]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

In [11]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [12]:
# Save references to each table
Station = Base.classes.station
Measurement = Base.classes.measurement

In [13]:
# Create our session (link) from Python to the DB
session=Session(engine)

# Exploratory Precipitation Analysis

In [15]:
Measurement.__table__.columns.keys()

['id', 'station', 'date', 'prcp', 'tobs']

In [16]:
Station.__table__.columns.keys()

['id', 'station', 'name', 'latitude', 'longitude', 'elevation']

In [17]:
# Find the most recent date in the data set.
recent_date= session.query(func.max(Measurement.date)).scalar()
recent_date

'2017-08-23'

In [ ]:
# Starting from the most recent data point in the database. 
end_date = dt.datetime.strptime(recent_date[0], '%Y-%m-%d').date()

# Calculate the date one year from the last date in data set.
start_date = end_date - relativedelta(months=12) 


df = pd.DataFrame([{"Date": x[0], "Precipitation": x[1]} for x in session.\
                                query(Measurement.date, Measurement.prcp).\
                                filter(Measurement.date >= start_date).\
                                order_by(Measurement.date).\
                                all() ])

df["Date"] = pd.to_datetime(df["Date"])
xticks = pd.DataFrame(df['Date'].unique(), columns=['Date'])

plt.bar(df['Date'], df["Precipitation"], width=5, label='Precipitation')

N = 39 
plt.xticks(xticks['Date'][::N], xticks['Date'][::N].dt.strftime('%Y-%m-%d'), rotation=90)
plt.xlabel("Date")
plt.ylabel("Inches")
plt.title('12 Months Precipitation Data', fontsize=15)
plt.legend()
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df['Precipitation'].describe().to_frame()

# Exploratory Station Analysis

In [29]:
Station.__table__.columns.keys()

['id', 'station', 'name', 'latitude', 'longitude', 'elevation']

In [31]:
# Design a query to calculate the total number of stations in the dataset
session.query(Station.station).distinct().count()

9

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
[x for x in session.
     query(Measurement.station, 
        func.count(Measurement.station))
        .group_by(Measurement.station)
        .order_by(desc(func.count(Measurement.station)))]

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
session.query(func.min(Measurement.tobs),
              func.max(Measurement.tobs), 
              func.avg(Measurement.tobs)).\
                filter(Measurement.station == "USC00519281").all()

In [ ]:
Measurement.__table__.columns.keys()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
args = [Measurement.date, Measurement.tobs]
station_temp =session.query(*args).filter(Measurement.station == "USC00519281",
                            Measurement.date >= start_date).\
                            all()
pd.DataFrame(station_temp, columns=['date', 'Temperature']).plot(kind='hist',figsize=(6,6), bins=12)
plt.title('USC00519281 Temp', fontsize=15)
plt.xlabel('Temperature')
plt.show()

# Close Session

In [ ]:
# Close Session
session.close()